### 2.3 PostgreSQL -  Manipulation de  données géométriques

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

<div style="font-size:120%" id="create">
Création d'une table comportant des champs géométriques :
</div>

In [2]:
c.execute('DROP TABLE IF EXISTS geotest;')

sql = '''CREATE TABLE geotest (
    id SERIAL PRIMARY KEY,
    name VARCHAR(64),
    pt POINT,
    l  LINE,
    s  LSEG,
    b  BOX,
    pa PATH,
    po POLYGON,
    c  CIRCLE 
);'''
c.execute(sql)
conn.commit()

### A - Points

<p style="font-size:120%" id="insert_point">
Enregistrement de points :
</p>

In [3]:
c.execute("DELETE FROM geotest WHERE pt IS NOT NULL;")

# SQL natif
c.execute("INSERT INTO geotest (name,pt) VALUES ('origin','(0,0)')")

# avec des arguments
args = ('target', (10,10))
c.execute("INSERT INTO geotest (name,pt) VALUES (%s,'%s')",args)

conn.commit()

<div style="font-size:120%" id="select_point">
Requêtage de points :
</div>

In [4]:
c.execute("SELECT id,name,pt FROM geotest")
data = c.fetchall()
for row in data:
    print(row)

[1, 'origin', '(0,0)']
[2, 'target', '(10,10)']


In [5]:
c.execute("SELECT id,name,pt FROM geotest WHERE pt ~= POINT '(0,0)'")
data = c.fetchall()
for row in data:
    print(row)

[1, 'origin', '(0,0)']


In [6]:
c.execute("SELECT id,name,pt FROM geotest WHERE pt[0] = 10")
data = c.fetchall()
for row in data:
    print(row)

[2, 'target', '(10,10)']


### B - Lignes

<p style="font-size:120%" id="insert_line">
Diverses syntaxes sont possibles pour l'expression littérale de lignes :
</p>

In [7]:
# lignes horizontales
c.execute("DELETE FROM geotest WHERE ?- l;")

# SQL natif
c.execute("INSERT INTO geotest (name,l) VALUES ('horizon','[(-100,0),(100,0)]')")
c.execute("INSERT INTO geotest (name,l) VALUES ('sky','(-100,100),(100,100)')")
c.execute("INSERT INTO geotest (name,l) VALUES ('cave','-100,-100,100,-100')")

conn.commit()

<div style="font-size:120%" id="insert_line_coefs">
Les lignes sont en fait des droites, représentées par trois valeurs A,B,C correspondant à l'équation&#160;:<div style="padding: 10px 0 0 40px">$Ax + By + C = 0$</div>
</div>

In [8]:
# lignes contenant le point (1,1)
c.execute("DELETE FROM geotest WHERE l <-> POINT(1,1) = 0;")

# ligne spécifiée via A,B,C
c.execute("INSERT INTO geotest (name,l) VALUES ('diagonal','{1,-1,0}')")
conn.commit()

<div style="font-size:120%" id="insert_line_dbapi">
Enregistrement de lignes avec des arguments à la DB-API&#160;:
</div>

In [9]:
# lignes verticales
c.execute("DELETE FROM geotest WHERE ?| l;")

# diagonale descendante
c.execute("DELETE FROM geotest WHERE name='descdiag';")


# avec des arguments
args = ('vertical',(0,100),(0,-100))
c.execute("INSERT INTO geotest (name,l) VALUES (%s,'[%s,%s]')",args)

args = ('left',(-100,100),(-100,-100))
c.execute("INSERT INTO geotest (name,l) VALUES (%s,'%s,%s')",args)

args = ('right',100,100,100,-100)
c.execute("INSERT INTO geotest (name,l) VALUES (%s,'%s,%s,%s,%s')",args)

args = ('descdiag',1,1,0)
c.execute("INSERT INTO geotest (name,l) VALUES (%s,'{%s,%s,%s}')",args)
conn.commit()

<div style="font-size:120%" id="select_line">
Les requêtes sur des lignes, renvoient A, B et C :
</div>

In [10]:
c.execute("SELECT id,name,l FROM geotest WHERE l IS NOT NULL")
data = c.fetchall()
for row in data:
    print(row)

[3, 'horizon', '{0,-1,0}']
[4, 'sky', '{0,-1,100}']
[5, 'cave', '{0,-1,-100}']
[6, 'diagonal', '{1,-1,0}']
[7, 'vertical', '{-1,0,0}']
[8, 'left', '{-1,0,-100}']
[9, 'right', '{-1,0,100}']
[10, 'descdiag', '{1,1,0}']


In [11]:
# les lignes sont des droites (sky est à distance nulle du point (1000,100))
args = ((1000,100),)
c.execute("SELECT id,name,l FROM geotest WHERE l <-> POINT '%s' = 0",args)
data = c.fetchone()
print(data)

[4, 'sky', '{0,-1,100}']


### C - Segments

<p style="font-size:120%" id="insert_segment">
Diverses syntaxes sont possibles pour l'enregistrement de segments&#160;:
</p>

In [12]:
c.execute("DELETE FROM geotest WHERE s IS NOT NULL")

# SQL natif
c.execute("INSERT INTO geotest (name,s) VALUES ('front_left',  '[(10, 2),(10,12)]' )")
c.execute("INSERT INTO geotest (name,s) VALUES ('front_top',   '((10,12),(20,12))' )")
c.execute("INSERT INTO geotest (name,s) VALUES ('front_right',  '(20,12),(20, 2)'  )")
c.execute("INSERT INTO geotest (name,s) VALUES ('front_bottom',  '20, 2 , 10, 2'   )")
conn.commit()

<div style="font-size:120%" id="insert_segment_dbapi">
Enregistrement de segments avec des arguments à la DB-API&#160;:
</div>

In [13]:
# avec arguments
args = ('top_left',(10,12),(15,17))
c.execute("INSERT INTO geotest (name,s) VALUES (%s,'[%s,%s]')",args)

args = ('top_right',(20,12),(25,17))
c.execute("INSERT INTO geotest (name,s) VALUES (%s,'(%s,%s)')",args)

args = ('bottom_right',(20,2),(25,7))
c.execute("INSERT INTO geotest (name,s) VALUES (%s,'%s,%s')",args)

args = ('top_rear',15,17,25,17)
c.execute("INSERT INTO geotest (name,s) VALUES (%s,'%s,%s,%s,%s')",args)

args = ('right_rear',(25,17),(25,7))
c.execute("INSERT INTO geotest (name,s) VALUES (%s,'%s,%s')",args)

conn.commit()

<div style="font-size:120%" id="request segment">
Requêtage de segments&#160;:
</div>

https://www.postgresql.org/docs/9.5/functions-geometry.html

In [14]:
c.execute("SELECT id,name,s FROM geotest WHERE s IS NOT NULL")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

11, front_left, [(10,2),(10,12)]
12, front_top, [(10,12),(20,12)]
13, front_right, [(20,12),(20,2)]
14, front_bottom, [(20,2),(10,2)]
15, top_left, [(10,12),(15,17)]
16, top_right, [(20,12),(25,17)]
17, bottom_right, [(20,2),(25,7)]
18, top_rear, [(15,17),(25,17)]
19, right_rear, [(25,17),(25,7)]


In [15]:
# segments horizontaux
c.execute("SELECT id,name,s FROM geotest WHERE ?- s")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

12, front_top, [(10,12),(20,12)]
14, front_bottom, [(20,2),(10,2)]
18, top_rear, [(15,17),(25,17)]


In [16]:
# segments verticaux
c.execute("SELECT id,name,s FROM geotest WHERE ?| s")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

11, front_left, [(10,2),(10,12)]
13, front_right, [(20,12),(20,2)]
19, right_rear, [(25,17),(25,7)]


In [17]:
# segments diagonaux
c.execute("SELECT id,name,s FROM geotest WHERE NOT ?| s AND NOT ?- s")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

15, top_left, [(10,12),(15,17)]
16, top_right, [(20,12),(25,17)]
17, bottom_right, [(20,2),(25,7)]


In [18]:
# segments passant par un point
c.execute("SELECT id,name,s FROM geotest WHERE s <-> POINT(10,12) = 0")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

11, front_left, [(10,2),(10,12)]
12, front_top, [(10,12),(20,12)]
15, top_left, [(10,12),(15,17)]


In [19]:
# les segments ne sont pas infinis
c.execute("SELECT id,name,s FROM geotest WHERE s <-> POINT '(30,2)' = 0")
print(c.fetchone())

None


### D - Chemins

<p style="font-size:120%" id="insert_open_path">
Enregistrement d'un chemin ouvert&#160;:
</p>

In [87]:
c.execute("DELETE FROM geotest WHERE pa IS NOT NULL")

# SQL natif
pts = [ (10,-10), (10,-15), (5,-15) ]
req = "INSERT INTO geotest (name,pa) VALUES ('open', '{}')".format(pts)
print(req)

c.execute(req)
conn.commit()

INSERT INTO geotest (name,pa) VALUES ('open', '[(10, -10), (10, -15), (5, -15)]')


In [88]:
# avec arguments
pts = [ (10,-12), (10,-17), (5,-17) ]
sql = "INSERT INTO geotest (name,pa) VALUES (%s, %s)"
args = ('open too', str(pts))
print(c.mogrify(sql,args).decode('UTF-8'))

c.execute(sql,args)
conn.commit()

INSERT INTO geotest (name,pa) VALUES ('open too', '[(10, -12), (10, -17), (5, -17)]')


In [89]:
c.execute("SELECT id,name,pa FROM geotest WHERE pa IS NOT NULL")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

50, open, [(10,-10),(10,-15),(5,-15)]
51, open too, [(10,-12),(10,-17),(5,-17)]


<p style="font-size:120%" id="insert_closed_path">
Enregistrement de chemins fermés&#160;:
</p>

In [90]:
c.execute("DELETE FROM geotest WHERE ISCLOSED(pa)")

# SQL natif
pts = ( (10,-14), (10,-19), (5,-19) )
req = "INSERT INTO geotest (name,pa) VALUES (%s, '{}')".format(pts)
print(req)

c.execute(req,('closed',))
conn.commit()

INSERT INTO geotest (name,pa) VALUES (%s, '((10, -14), (10, -19), (5, -19))')


In [91]:
# avec arguments
pts = ( (10,-16), (10,-21), (5,-21) )
sql = "INSERT INTO geotest (name,pa) VALUES (%s, %s)"
args = ('closed too', str(pts))
print(c.mogrify(sql,args).decode('UTF-8'))

c.execute(sql,args)
conn.commit()

INSERT INTO geotest (name,pa) VALUES ('closed too', '((10, -16), (10, -21), (5, -21))')


In [92]:
# variantes
pts = ( (10,-18), (10,-23), (5,-23) )
sql = "INSERT INTO geotest (name,pa) VALUES (%s, %s)"
args = ('closed until tomorrow', ', '.join([str(p) for p in pts]))

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

pts = ( (10,-20), (10,-25), (5,-25) )
sql = "INSERT INTO geotest (name,pa) VALUES (%s, %s)"
args = ('closed for season', ', '.join(['{},{}'.format(*p) for p in pts]) )

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

conn.commit()

INSERT INTO geotest (name,pa) VALUES ('closed until tomorrow', '(10, -18), (10, -23), (5, -23)')
INSERT INTO geotest (name,pa) VALUES ('closed for season', '10,-20, 10,-25, 5,-25')


<p style="font-size:120%" id="request_path">
Requêtage de chemins&#160;:
</p>

In [93]:
# chemins fermés
c.execute("SELECT id,name,pa FROM geotest WHERE ISCLOSED(pa)")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

52, closed, ((10,-14),(10,-19),(5,-19))
53, closed too, ((10,-16),(10,-21),(5,-21))
54, closed until tomorrow, ((10,-18),(10,-23),(5,-23))
55, closed for season, ((10,-20),(10,-25),(5,-25))


In [94]:
# chemins ouverts
c.execute("SELECT id,name,pa FROM geotest WHERE ISOPEN(pa)")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

50, open, [(10,-10),(10,-15),(5,-15)]
51, open too, [(10,-12),(10,-17),(5,-17)]


In [100]:
# conversion en polygone
c.execute("SELECT POLYGON(pa) FROM geotest WHERE name='closed'")
output = str(c.fetchone()[0])

assert output == '((10,-14),(10,-19),(5,-19))'

### E - Cercles
<p style="font-size:120%" id="insert_circle">
Enregistrement de cercles&#160;:
</p>

In [116]:
c.execute("DELETE FROM geotest WHERE c IS NOT NULL")

# expression littérale '<(-20, 20), 2.5>'
circ = ( (-20,20), 2.5)
sql = "INSERT INTO geotest (name,c) VALUES (%s, %s)"
args = ('c1', '<{}>'.format(', '.join([str(p) for p in circ])))

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

# expression littérale '((-20, 20), 5)'
circ = ( (-20,20), 5)
sql = "INSERT INTO geotest (name,c) VALUES (%s, %s)"
args = ('c2', str(circ))

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

# expression littérale '(-20, 20), 7.5'
circ = ( (-20,20), 7.5)
sql = "INSERT INTO geotest (name,c) VALUES (%s, %s)"
args = ('c3', ', '.join([str(v) for v in circ]))

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

# expression littérale '-20, 20, 10'
circ = ( (-20,20), 10)
sql = "INSERT INTO geotest (name,c) VALUES (%s, %s)"
args = ('c4', ', '.join([str(v) for v in (*circ[0],circ[1])]))

print(c.mogrify(sql,args).decode('UTF-8'))
c.execute(sql,args)

conn.commit()

INSERT INTO geotest (name,c) VALUES ('c1', '<(-20, 20), 2.5>')
INSERT INTO geotest (name,c) VALUES ('c2', '((-20, 20), 5)')
INSERT INTO geotest (name,c) VALUES ('c3', '(-20, 20), 7.5')
INSERT INTO geotest (name,c) VALUES ('c4', '-20, 20, 10')


<p style="font-size:120%" id="select_circle">
Requêtage de cercles&#160;:
</p>

In [117]:
c.execute("SELECT id,name,c FROM geotest WHERE c IS NOT NULL")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

74, c1, <(-20,20),2.5>
75, c2, <(-20,20),5>
76, c3, <(-20,20),7.5>
77, c4, <(-20,20),10>


In [126]:
c.execute("SELECT id,name,c FROM geotest WHERE DIAMETER(c) > 10 ")
for d in c.fetchall(): print(', '.join([str(v) for v in d]))

76, c3, <(-20,20),7.5>
77, c4, <(-20,20),10>
